In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import uuid
import mlflow
import optuna
import mlflow.sklearn
from catboost import CatBoostRegressor
from mlflow.tracking import MlflowClient
from mlflow.models import infer_signature
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from typing import Dict, Union
from sklearn.preprocessing import StandardScaler

# Starting ML Tracking Server

In the terminal, navigate to the notebook folder and the start the ml server

Run in the terminal (command line): mlflow server --host 127.0.0.1 --port 8080

**This is what it does:**

1. Starts the MLflow Tracking Server
A dedicated process that manages and serves your MLflow experiments.

2. Provides a Web UI
Accessible at http://127.0.0.1:8080 (or localhost:8080), where you can browse experiments, runs, parameters, metrics, and artifacts.

3. Exposes a Tracking API Endpoint
Other scripts or notebooks can log directly to this server if you set - **mlflow.set_tracking_uri("http://127.0.0.1:8080")**

4. MLflow automatically creates a folder mlruns/ in your working directory the first time you log something.
Inside mlruns/, it creates subfolders for:

- each experiment (default is 0)

- each run within that experiment

# Setting URI

In [2]:
# Set the tracking uri
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

What it does:

1. It tells your MLflow client (your script/notebook) where to send all logging data (experiments, runs, params, metrics, artifacts).

3. Since our host is local, it will still write to mlruns, but here you could configure a remote host URI.

In [3]:
print("Current Tracking URI:", mlflow.get_tracking_uri())

Current Tracking URI: http://127.0.0.1:8080


# Fitting the model

Prepare the data, split it into training, validation, and test sets, and train a simple LASSO model with a regularization hyperparameter of alpha = 0.1.

In [6]:
# Load datasets
df = pd.read_csv('WineQT.csv', sep=',')

# Split the data into training, validation, and test sets
train, test = train_test_split(df, test_size=0.25, random_state=42)
x_train = train.drop(["quality"], axis=1).values
y_train = train[["quality"]].values.ravel()
x_test= test.drop(["quality"], axis=1).values
y_test = test[["quality"]].values.ravel()

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

# Setting hyperparameter
alpha = 0.1
model = Lasso(alpha=alpha)
model.fit(x_train, y_train)

# Predicting
y_pred_val = model.predict(x_val)
y_pred_test = model.predict(x_test)

# Calculating metrics
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'RMSE val = {rmse_val}, RMSE test = {rmse_test}')

RMSE val = 0.6356127117266147, RMSE test = 0.638554128638353


What we have just done is we conducted an experimental run! We set a hyperparameter, trained a model, and got validation and test scores (happy face). Now, our job is to log this to MLflow.

# Running first experiment

To do that, we need to run the command:

mlflow.set_experiment("Wine Quality Experiment")

If we do so, in the UI (http://127.0.0.1:8080), we will see that the experiment is created.

In [7]:
# Set the experiment name - it also creates an experiment if it doesn't exist
mlflow.set_experiment("Wine Quality Regression")

<Experiment: artifact_location='mlflow-artifacts:/677703609984501005', creation_time=1756279706345, experiment_id='677703609984501005', last_update_time=1756279706345, lifecycle_stage='active', name='Wine Quality Regression', tags={}>

Now, all we need to do is log the parameters, metrics, and the model.

In [8]:
# Setting hyperparameters
params = {
    'alpha': model.alpha
}

# Start an MLflow run
with mlflow.start_run(run_name='lasso_baseline'):
    # Log the hyperparameters
    mlflow.log_params(params)
    # Log the loss metric
    mlflow.log_metric("rmse_val", rmse_val)
    mlflow.log_metric("rmse_test", rmse_test)
    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("model_version", "baseline")

    model_name = "wine_model_lasso"
    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        name=model_name,
        input_example=x_train,
    )

🏃 View run lasso_baseline at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/b7e37b95c783488d9fbaddf0986074d3
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005


You can click the links above to see the run and experiment in the Tracking Server UI.

# Running MLflow experiments with different models

Now, let's make our code a bit more modular, so we can efficiently run experiments for different models without code repetition. We make 2 functions: train_model and log_run.

In [9]:
def train_model(df: pd.DataFrame, params: dict, model='Lasso'):
    """
    Train a model and return the metrics
    """
    # Split the data into training, validation, and test sets
    train, test = train_test_split(df, test_size=0.25, random_state=42)
    x_train = train.drop(["quality"], axis=1).values
    y_train = train[["quality"]].values.ravel()
    x_test= test.drop(["quality"], axis=1).values
    y_test = test[["quality"]].values.ravel()

    # Split the data into training and validation sets
    x_train, x_val, y_train, y_val = train_test_split(
        x_train, y_train, test_size=0.2, random_state=42
    )

    if model == 'Lasso':
        model = Lasso(**params)
    elif model == 'CatBoost':
        model = CatBoostRegressor(**params)
    else:
        raise ValueError(f"Model {model} not supported")

    # Fitting the model
    model.fit(x_train, y_train)
    # Predicting
    y_pred_val = model.predict(x_val)
    y_pred_test = model.predict(x_test)

    # Calculating metrics
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    print(f'RMSE val = {rmse_val}, RMSE test = {rmse_test}')
    return model,rmse_val, rmse_test

In [10]:
def log_run(
        run_name: str,
        params: dict,
        metrics: dict,
        tags: dict,
        trained_model,
        model_type='Lasso',
        input_example=None,
        registered_model_name=None):
    """
    Log a run to MLflow
    """
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    mlflow.set_tag("model_version", run_name)
    mlflow.set_tags(tags)

    if model_type == 'Lasso':
        # Log sklearn model
        model_info = mlflow.sklearn.log_model(
            sk_model=trained_model,
            name="wine_model_lasso",
            input_example=input_example,
            registered_model_name=registered_model_name,
        )
    elif model_type == 'CatBoost':
        # Log CatBoost model
        model_info = mlflow.catboost.log_model(
            cb_model=trained_model,
            name="wine_model_catboost",
            input_example=input_example,
            registered_model_name=registered_model_name,
        )
    else:
        raise ValueError(f"Model type {model_type} not supported")
    
    return model_info

### Running LASSO

In [11]:
# Setting hyperparameters

lasso_params = {
    'alpha': 0.1,
    'max_iter': 1000,
    'random_state': 42
}

# Train the model
trained_model, rmse_val, rmse_test = train_model(df, lasso_params, model='Lasso')

# Start an MLflow run
with mlflow.start_run(run_name='lasso_baseline'):
    # Prepare metrics and tags
    metrics = {
        'rmse_val': rmse_val,
        'rmse_test': rmse_test
    }
    
    # Let's add some tags to the run
    tags = {
        'model_version': 'baseline',
        'experiment_type': 'regression',
    }
    
    # Log everything using our function
    model_info = log_run(
        run_name="lasso_baseline",
        params=lasso_params,
        metrics=metrics,
        tags=tags,
        trained_model=trained_model,
        model_type='Lasso',
    )

RMSE val = 0.6356127117266147, RMSE test = 0.638554128638353


2025/08/27 12:12:38 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run lasso_baseline at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/d71ef0989d044ea88e66c7cf1384ae25
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005


 Now, let's run and log an experiment with a CatBoost model.

In [12]:
# Setting hyperparameters for CatBoost
catboost_params = {
    'iterations': 100,
    'learning_rate': 0.1,
    'depth': 6,
    'random_seed': 42,
    'verbose': False
}

# Train the model
trained_model, rmse_val, rmse_test = train_model(df, catboost_params, model='CatBoost')

# Start an MLflow run
with mlflow.start_run(run_name='catboost_baseline'):
    # Prepare metrics and tags
    metrics = {
        'rmse_val': rmse_val,
        'rmse_test': rmse_test
    }
    
    tags = {
        'model_version': 'baseline',
        'experiment_type': 'regression',
        'data_version': 'v1',
        'algorithm': 'gradient_boosting'
    }
    
    # Log everything using our function
    model_info = log_run(
        run_name="catboost_baseline",
        params=catboost_params,
        metrics=metrics,
        tags=tags,
        trained_model=trained_model,
        model_type='CatBoost',
    )

RMSE val = 0.5558687336128091, RMSE test = 0.5729224664412881


2025/08/27 12:12:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run catboost_baseline at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/486d46f4b7ce4f49ae6a96bee7e25fd0
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005


Now, i the UI (http://127.0.0.1:8080/#/experiments) we can compare 3 different experimental runs - 2 LASSO and 1 CatBoost. To do that, select all the runs and press Compare as we have done above. Since the parameters are different for the models, the visual comparison of parameters might not tell much. However, we can compare the validation and test RMSE.

# Searching for the best model

Up to now, we could find the best model manually using the UI, which is great. However, when developing ML pipelines and registering the best model to be deployed, we need to be able to find this model programmatically. Here's how to do that.

First, we need to find the experiment ID for which we want to select the best models. We can do that using search_experiments() method.

In [13]:
# Get experiment by name
experiment_name = "Wine Quality Regression"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment:
    experiment_id = experiment.experiment_id
    print(f"Experiment '{experiment_name}' has ID: {experiment_id}")
else:
    print(f"Experiment '{experiment_name}' not found")

Experiment 'Wine Quality Regression' has ID: 677703609984501005


As we know the experiment ID, we can check what the top models are for this experiment and select the best model. To select the best model, we can just select the one at the top row of the resulting top_models dataframe. In our case, this is the CatBoost model.

In [18]:
# Find high-performing models across experiments
top_models = mlflow.search_logged_models(
    experiment_ids=[str(experiment_id)],
    filter_string="metrics.rmse_test < 0.9",
    order_by=[{"field_name": "metrics.rmse_val", "ascending": True}],
)
best_model = top_models.iloc[0]
best_model

artifact_location         mlflow-artifacts:/677703609984501005/models/m-...
creation_timestamp                                            1756282364254
experiment_id                                            677703609984501005
last_updated_timestamp                                        1756282365608
metrics                   [<Metric: dataset_digest=None, dataset_name=No...
model_id                                 m-62cd8f0cdcbc4fdca59cf6c46b7e3379
model_type                                                                 
name                                                    wine_model_catboost
params                    {'depth': '6', 'learning_rate': '0.1', 'verbos...
source_run_id                              486d46f4b7ce4f49ae6a96bee7e25fd0
status                                                                READY
status_message                                                             
tags                      {'mlflow.user': 'timurbikmuhametov', 'mlflow.s...
Name: 0, dty

In [19]:
# Load the best model for inference
loaded_model = mlflow.pyfunc.load_model(f"models:/{best_model.model_id}")
y_pred = loaded_model.predict(x_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE on test set: {rmse_test}")

RMSE on test set: 0.5729224664412881


## Child runs

MLflow tracks experiments as named groups where all your related runs live. As we have seen up to now, each "run" is one training session where you log parameters, metrics, and artifacts. Parent and Child Runs add a hierarchical layer to this setup.

**Why Should You Care?**

1. Organizational Clarity.

With a parent-child structure, related runs are automatically grouped together. When you're running a hyperparameter search using a Bayesian approach on a particular model architecture, every iteration gets logged as a child run, while the overarching Bayesian optimization process becomes the parent run. This eliminates the guesswork of figuring out which runs belong together.

2. Enhanced Traceability.

When working on complex projects with multiple variants, child runs represent individual products or model variations. This makes it straightforward to trace back results, metrics, or artifacts to their specific run configuration. Need to find that exact setup that gave you the best performance? Just follow the hierarchy.

3. Improved Scalability.

As your experiments grow in number and complexity, having a nested structure ensures your tracking remains manageable. Navigating through a structured hierarchy is much more efficient than scrolling through a flat list of hundreds or thousands of runs. This becomes particularly valuable as projects scale up.

Now, as the concept of Parent and Child runs is clear, let's see how we can implement this.

 

### Grid search for LASSO

Let me show you how this works with our LASSO example. Say you're testing a LASSO model with different alpha values - from 1 to 0.001.

To do that, we create a function log_run_child.

This is a very similar function to log_run, but here we change the names of the run at each run and also specify parameter nested=True. This parameter indicates to MLflow that the runs are child runs.

Then we specify the parent run, and in a loop we run the child runs.

In [20]:
def log_run_child(
        run_name: str,
        params: dict,
        metrics: dict,
        tags: dict,
        trained_model,
        model_type='Lasso',
        input_example=None,
        registered_model_name=None,
        iteration=None
        ):
    """
    Log a run to MLflow
    """
    with mlflow.start_run(run_name=f"{run_name}_iteration_{iteration}", nested=True):
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)
        mlflow.set_tag("model_version", run_name)
        mlflow.set_tags(tags)

        if model_type == 'Lasso':
            # Log sklearn model
            model_info = mlflow.sklearn.log_model(
                name=f"{run_name}_iteration_{iteration}_wine_model_lasso",
                sk_model=trained_model,
                input_example=input_example,
                registered_model_name=registered_model_name,
            )
        elif model_type == 'CatBoost':
            # Log CatBoost model
            model_info = mlflow.catboost.log_model(
                name=f"{run_name}_iteration_{iteration}_wine_model_catboost",
                cb_model=trained_model,
                input_example=input_example,
                registered_model_name=registered_model_name,
            )
        else:
            raise ValueError(f"Model type {model_type} not supported")
    
    return model_info

In [22]:
# Specify several alphas
alpha_range = np.logspace(0, -3, num=10).tolist()
run_name = 'Lasso_child_runs_1_0_point_001'

with mlflow.start_run(run_name=run_name):
    for idx, alpha in enumerate(alpha_range):
        params = {'alpha': alpha}
        # Train the model
        trained_model, rmse_val, rmse_test = train_model(df, params, model='Lasso')
        # Prepare metrics
        metrics = {'rmse_val': rmse_val, 'rmse_test': rmse_test}
    # Log everything using our function
        model_info = log_run_child(
            run_name=run_name,
            params=params,
            metrics=metrics,
            tags=tags,
            trained_model=trained_model,
            model_type='Lasso',
            iteration=idx
        )

RMSE val = 0.7546000742808262, RMSE test = 0.7374379792573892


2025/08/27 12:29:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Lasso_child_runs_1_0_point_001_iteration_0 at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/2b23fea4928e45f589818b9195f6893e
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005
RMSE val = 0.7541178879815691, RMSE test = 0.7366319489063405


2025/08/27 12:29:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Lasso_child_runs_1_0_point_001_iteration_1 at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/6b4fa35858d94675b330377776bca249
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005
RMSE val = 0.6762060849767165, RMSE test = 0.6743324662326164


...

🏃 View run Lasso_child_runs_1_0_point_001_iteration_7 at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/434d538010544e48bc8ac7320fdd9413
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005
RMSE val = 0.5726741757910615, RMSE test = 0.6072882839308418


2025/08/27 12:29:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Lasso_child_runs_1_0_point_001_iteration_8 at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/312cc97c6ee84949a2687235d0d80a97
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005
RMSE val = 0.5713198123541341, RMSE test = 0.6098083649853254


2025/08/27 12:29:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Lasso_child_runs_1_0_point_001_iteration_9 at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/2f622e286947475f9e923d00e407b05a
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005
🏃 View run Lasso_child_runs_1_0_point_001 at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/2b990ac2f7d84084a78fda542eea08ed
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005


### Child runs with Optuna

In [23]:
def objective(
    trial: optuna.Trial, 
    x_train: np.ndarray, 
    y_train: np.ndarray, 
    x_val: np.ndarray, 
    y_val: np.ndarray, 
    x_test: np.ndarray, 
    y_test: np.ndarray
) -> float:
    np.random.seed(42)
    """Optimize CatBoost hyperparameters using Optuna."""
    # Sample CatBoost hyperparameters (fixed iterations for faster trials)
    params: Dict[str, Union[int, float, bool]] = {
        'iterations': 100,  # Fixed iterations for trials
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'random_seed': 42,
        'verbose': False
    }
    
    model: CatBoostRegressor = CatBoostRegressor(**params)
    model.fit(x_train, y_train)
    
    # Predictions
    y_pred_val: np.ndarray = model.predict(x_val)
    y_pred_test: np.ndarray = model.predict(x_test)
    
    # Errors
    rmse_val: float = np.sqrt(mean_squared_error(y_val, y_pred_val))
    rmse_test: float = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    # Log the run with log_run_child
    log_run_child(
        run_name=run_name,
        params=params,
        metrics={'rmse_val': rmse_val, 'rmse_test': rmse_test},
        tags={'model_version': 'catboost_optuna', 'trial': trial.number},
        trained_model=model,
        model_type='CatBoost',
        iteration=trial.number
    )
    
    return round(rmse_val, 3)

In [28]:
run_name = 'Catboost_optuna'
with mlflow.start_run(run_name=run_name):
    np.random.seed(42)
    # Create the study and optimize
    study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(
        lambda trial: objective(trial, x_train, y_train, x_val, y_val, x_test, y_test),
        n_trials=50
    )
    
    # Log the best metrics and parameters from the parent run
    mlflow.log_metric("best_rmse", study.best_value)
    mlflow.log_params(study.best_params)
    mlflow.set_tag('best_model', 'true')
    
    # Train the best model with early stopping (more iterations)
    best_params = study.best_params.copy()
    best_params.update({
        'iterations': 1000,  # More iterations for final model
        'early_stopping_rounds': 50,  # Early stopping
    })
    
    best_model = CatBoostRegressor(**best_params, random_seed=42)
    best_model.fit(x_train, y_train, eval_set=[(x_val, y_val)], verbose=False)
    
    # Compute test score for the best model
    y_pred_test = best_model.predict(x_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    
    # Log the test score
    mlflow.log_metric("rmse_test_best", rmse_test)
    
    # Log the best model in the parent run
    model_info = mlflow.sklearn.log_model(
        sk_model=best_model,
        name="best_catboost_model",
        input_example=x_train,
        tags={'best_model': 'true'}
    )
    
    print(f"Best validation RMSE: {study.best_value:.4f}")
    print(f"Test RMSE: {rmse_test:.4f}")

[I 2025-08-27 12:45:04,344] A new study created in memory with name: no-name-b318f382-8968-4262-a3bb-7076cfca9dd2
2025/08/27 12:45:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-27 12:45:06,307] Trial 0 finished with value: 0.596 and parameters: {'learning_rate': 0.03574712922600244, 'depth': 10, 'l2_leaf_reg': 7.587945476302646}. Best is trial 0 with value: 0.596.


🏃 View run Catboost_optuna_iteration_0 at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/2c626324209446a78e8ee5eea3ed66e6
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005


2025/08/27 12:45:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-27 12:45:07,610] Trial 1 finished with value: 0.574 and parameters: {'learning_rate': 0.07661100707771368, 'depth': 5, 'l2_leaf_reg': 2.403950683025824}. Best is trial 1 with value: 0.574.


🏃 View run Catboost_optuna_iteration_1 at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/4444b476f0ca4abfab158e9ee50414ac
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005


2025/08/27 12:45:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-27 12:45:09,236] Trial 2 finished with value: 0.653 and parameters: {'learning_rate': 0.012184186502221764, 'depth': 10, 'l2_leaf_reg': 6.41003510568888}. Best is trial 1 with value: 0.574.


...

🏃 View run Catboost_optuna_iteration_47 at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/4912c65d76224019b0373bea42cfe4c7
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005


2025/08/27 12:46:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-27 12:46:13,304] Trial 48 finished with value: 0.566 and parameters: {'learning_rate': 0.24549414786515986, 'depth': 9, 'l2_leaf_reg': 5.495720049674533}. Best is trial 43 with value: 0.551.


🏃 View run Catboost_optuna_iteration_48 at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/281abccbd8c445a2b3bcf3fbd297e992
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005


2025/08/27 12:46:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-08-27 12:46:14,663] Trial 49 finished with value: 0.568 and parameters: {'learning_rate': 0.07842255349941261, 'depth': 8, 'l2_leaf_reg': 3.215662322272498}. Best is trial 43 with value: 0.551.


🏃 View run Catboost_optuna_iteration_49 at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/cff1fc5e17c445239d1a50139aa4c565
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005
Best validation RMSE: 0.5510
Test RMSE: 0.5768
🏃 View run Catboost_optuna at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/7517e09c15974f5b92eb030c9d9f268d
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005


# Loading the best optuna model

Now, note that when we run the parent run, we re-train the best optuna model on the entire training set and log a tag - best_mode: true. This allows us to easily load the best model, use it for predictions and consider moving the model to production.

In [37]:
# Search across all logged models for the tag
top_models = mlflow.search_logged_models(
    filter_string="tags.best_model = 'true'",
    order_by=[{"field_name": "last_updated_timestamp", "ascending": False}],
    max_results=1
)

# Get the latest "best_model"
best_model = top_models.iloc[0]

# Extract the model URI
model_uri = best_model['model_id']

# Load it
loaded_model = mlflow.pyfunc.load_model(f'models:/{model_uri}')

# Use it
y_pred = loaded_model.predict(x_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE on test set: {rmse_test}")

RMSE on test set: 0.5767685743260196


# Beyond basics - what do you need to log to MLflow?

In [41]:
# Load datasets
df = pd.read_csv('WineQT.csv', sep=',')

# Split the data into training, validation, and test sets
train, test = train_test_split(df, test_size=0.25, random_state=42)
x_train = train.drop(["quality"], axis=1).values
y_train = train[["quality"]].values.ravel()
x_test= test.drop(["quality"], axis=1).values
y_test = test[["quality"]].values.ravel()

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

### Signatures and input examples

In [ ]:
# Setting hyperparameters for CatBoost
catboost_params = {
    'iterations': 100,
    'learning_rate': 0.1,
    'depth': 6,
    'random_seed': 42,
    'verbose': False
}

# Train the model and make predictions
model = CatBoostRegressor(**catboost_params)
model.fit(x_train, y_train)
y_pred_test = model.predict(x_test)

# Infer signature from data
signature = infer_signature(x_test, y_pred_test)
input_example = x_train[:5]

# Start an MLflow run
with mlflow.start_run(run_name='catboost_with_signature'):
    
    # Log CatBoost model with model example and signature
        model_info = mlflow.catboost.log_model(
            cb_model=trained_model,
            name="wine_model_catboost",
            input_example=input_example,
            signature=signature
        )


inputs: 
  [Tensor('float64', (-1, 12))]
outputs: 
  [Tensor('float64', (-1,))]
params: 
  None

### Artifacts

In [ ]:
# --- Single parameter for Lasso
alpha = 0.01  

# --- Fit scaler separately
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled  = scaler.transform(x_test)

# --- Fit Lasso
model = Lasso(alpha=alpha, random_state=42, max_iter=10000)
model.fit(X_train_scaled, y_train)
y_pred_test = model.predict(X_test_scaled)

# --- Signature & input example (raw features as input)
signature = infer_signature(x_test, y_pred_test)
input_example = x_train[:5]

# --- Local paths for artifacts
run_tmp = Path("mlflow_local_artifacts") / str(uuid.uuid4())
run_tmp.mkdir(parents=True, exist_ok=True)
scaler_path = run_tmp / "standard_scaler.pkl"
plot_path   = run_tmp / "pred_vs_true.png"

# Save scaler separately
joblib.dump(scaler, scaler_path)


🏃 View run lasso_artifacts at: http://127.0.0.1:8080/#/experiments/677703609984501005/runs/48e95d95802e470baf9423d98de8170a
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/677703609984501005


In [ ]:
# --- Plot Predicted vs True
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred_test, alpha=0.6)
plt.xlabel("True values")
plt.ylabel("Predicted values")
plt.title("Predicted vs. True (Lasso)")
axis_min = min(np.min(y_test), np.min(y_pred_test))
axis_max = max(np.max(y_test), np.max(y_pred_test))
plt.plot([axis_min, axis_max], [axis_min, axis_max], linestyle="--")
plt.tight_layout()
plt.savefig(plot_path, dpi=150)
plt.close()

# --- MLflow logging
with mlflow.start_run(run_name="lasso_artifacts"):
    # Log artifacts (scaler + plot)
    mlflow.log_artifact(str(scaler_path), artifact_path="artifacts")
    mlflow.log_artifact(str(plot_path),   artifact_path="artifacts")

    # Log Lasso model (trained on scaled inputs)
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        name="wine_model_lasso",
        signature=signature,
        input_example=input_example
    )